In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 50 # how many independent sequences will we process in parallel?
block_size = 256  # Увеличение максимальной длины контекста
 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384 #CHANGE #2 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
n_head = 8
n_layer = 6
dropout = 0.1 #CHANGE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! 
# ------------
from transformers import BertTokenizer

#CHANGE #3 IS second encoding
import tiktoken 

# Загрузка словаря p50k_base
# tokenizer = tiktoken.get_encoding("p50k_base")

# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")



# Загружаем предобученный токенизатор BERT
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('train.txt', 'r', encoding='utf-8') as f:
    train = f.read()

with open('test.txt', 'r', encoding='utf-8') as f:
    test = f.read()

    # with open('valid.txt', 'r', encoding='utf-8') as f:
    # val_Data = f.read()


# here are all the unique characters that occur in this text
chars_train = sorted(list(set(train)))
chars_test = sorted(list(set(test)))
number_of_unique_tokens_train = len(chars_train)
number_of_unique_tokens_test = len(chars_test)
vocab_size = len(chars_train)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars_train) }
itos = { i:ch for i,ch in enumerate(chars_train) }


encode = lambda s: [stoi[c] for c in s]  # Кодирование
decode = lambda l: ''.join([itos[i] for i in l])
  # Декодирование

# Train and test splits

train_data = encode(train)


test_data = encode(test)


train_dataset = torch.tensor(train_data, dtype=torch.long)
# train_dataset = train_dataset.to(device)
test_dataset = torch.tensor(test_data, dtype=torch.long)
# test_dataset = test_dataset.to(device)


# n = int(0.8*len(data)) # first 90% will be train, rest val
# train_data = torch.tensor(data[:n], dtype=torch.long)
# val_data = torch.tensor(data[n:],dtype=torch.long)

# unique_tokens = list(set(data))
# number_of_unique_tokens = len(unique_tokens)



print(f"VOCAB SIZE: {vocab_size}")
print(f"Number of unique tokens for train: {number_of_unique_tokens_train}")
print(f"Number of unique tokens for test: {number_of_unique_tokens_test}")






# print("NUMBER OF UNIQUE TOKENS:",number_of_unique_tokens)

# print("NUMBER OF OVERALL TOKENS:", vocab_size)



# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_dataset if split == 'train' else test_dataset
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'test']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['test']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model




C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# generate from the model
encoded_tokens = encode("Hello my friend")

  # Предположим, что encode возвращает список токенов
input_tensor = torch.tensor(encoded_tokens)

# Добавление нового измерения для создания двумерного тензора
input = input_tensor.unsqueeze(0)  # Добавляет новое измерение спереди, делая тензор двумерным
input = input.to(device)

# generate from the model

print(decode(m.generate(input, max_new_tokens=500)[0].tolist()))
# open('more.txt', 'w').write(decode(m.generate(input, max_new_tokens=10000)[0].tolist()))


Hello my friend and a men . It is interded out if he with who possibly order Museum Southern inhibitionis and protagasa : " In to evisual congress to Winlvan Read and Techman Senat of UKA had said who campaigned the Taxe can sign zine <unk> . " Cynnests the Ottoma Museum Trevolud contribution on June 50D and an attem time compary volum the home to exercise imply homelitize and army lafley . " The Zealer wills , eso remassed in Marro the fort of Trammes and Molelowork 's thirtheved to win — she is bushin , brow


Device 0: NVIDIA GeForce RTX 3050 Laptop GPU
Memory Allocated: 3.17 GB
Memory Reserved: 3.72 GB
Memory Free: 0.57 GB
Total Memory: 4.29 GB


11.67869 M parameters
step 0: train loss 7.3427, test loss 7.3428
step 500: train loss 1.9456, test loss 1.9347
step 1000: train loss 1.5608, test loss 1.5445
step 1500: train loss 1.4386, test loss 1.4185
step 2000: train loss 1.3760, test loss 1.3546
step 2500: train loss 1.3423, test loss 1.3211
step 3000: train loss 1.3165, test loss 1.2913
step 3500: train loss 1.2960, test loss 1.2759
step 4000: train loss 1.2736, test loss 1.2500
step 4500: train loss 1.2568, test loss 1.2342
step 4999: train loss 1.2437, test loss 1.2178


Hello, how are you? later previous
